In [1]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim, os, pickle
import tokenize
import numpy as np
import pandas as pd

In [2]:
##### take some data
df_train = pd.read_csv('../../../../Data/Raw/train_clean.csv',encoding='utf-8')
qid_stack = df_train['qid1'].append(df_train['qid2'])
df_train_naExc = df_train[~df_train.isnull().any(axis=1)]
df_buf1 = df_train_naExc[['qid1','question1']]
df_buf2 = df_train_naExc[['qid2','question2']]
df_buf1.columns = ['qid','question']
df_buf2.columns = ['qid','question']
df_stack = df_buf1.append(df_buf2)
df_stack_dupExc = df_stack.drop_duplicates()
test_buf = df_stack_dupExc['question']#.head(1000)#.iloc[5000:7000]#head(1000)

In [3]:
pkl_file = open('LDA_pickle_New_20170520_50topic_ldamodel_test_traing.pkl', 'rb')
ldamodel_dump = pickle.load(pkl_file)
pkl_file.close()

pkl_file1 = open('Py2_dictionary.pkl', 'rb')
dictionary = pickle.load(pkl_file1)
pkl_file.close()
#print(ldamodel_dump.keys())
##### for predict 
#ldamodel = ldamodel_dump['lda_model']
#dictionary = ldamodel_dump['dictionary']
##### for build the lda-model by yourself 
#corpus = ldamodel_dump['encoding_corpus']

In [4]:
############# some tools
######### Tokenize the corpus
tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
def tokenize_doc(new_corpus):
    new_corpus
    texts = []
    ###
    ### Progressing bar setting
    totall_size = len(new_corpus)
    counter = 0
    progress = 0
    # loop through document list
    for i in new_corpus:    
#         ### Monitor the progress
#         if progress%(1000) ==0:
#             bar = "[" + int(progress/1000)*"|"+">"+ (10-int(progress/1000))*"-" + str((progress/100)) + "%" "]"
#             print(bar)
#         #### count
#         counter += 1
#         progress = int(10000*(counter/totall_size))

        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)

        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # stem tokens
#         stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        stemmed_tokens = []
        for i in stopped_tokens:
            try:
                stemmed_tokens = stemmed_tokens + [p_stemmer.stem(i)]
            except:
                print(i)
        # add tokens to list            
        texts.append(stemmed_tokens)
    return texts

######  predict the topic
def topic_predict(new_corpus, model, dictionary):
    tokenize_buf = tokenize_doc([new_corpus])
    doc_bow = [dictionary.doc2bow(text) for text in tokenize_buf]
    que_vec = [item for itemList in doc_bow for item in itemList]
    topic_vec = model[que_vec]

    word_count_array = np.empty((len(topic_vec), 2), dtype = np.object)
    for i in range(len(topic_vec)):
        word_count_array[i, 0] = topic_vec[i][0]
        word_count_array[i, 1] = topic_vec[i][1]

    idx = np.argsort(word_count_array[:, 1])
    idx = idx[::-1]
    word_count_array = word_count_array[idx]

    final = []
    final = model.print_topic(20, 20)# model.print_topic(word_count_array[0, 0], len(word_count_array)) # model.print_topic(word_count_array[0, 0], len(word_count_array))#1)

    question_topic = final.split('*') ## as format is like "probability * topic"
    return question_topic

In [61]:
###### check the topic stack
print(ldamodel_dump.print_topics(num_topics=10, num_words=4))

[(27, u'0.231*"peopl" + 0.104*"think" + 0.050*"import" + 0.034*"american"'), (5, u'0.067*"state" + 0.046*"china" + 0.036*"unit" + 0.027*"polit"'), (28, u'0.066*"math" + 0.064*"2" + 0.061*"c" + 0.055*"market"'), (34, u'0.054*"man" + 0.039*"choos" + 0.037*"purpos" + 0.035*"girlfriend"'), (8, u'0.053*"cultur" + 0.040*"differ" + 0.032*"group" + 0.029*"convert"'), (17, u'0.047*"trump" + 0.047*"2016" + 0.045*"come" + 0.043*"product"'), (7, u'0.096*"phone" + 0.039*"social" + 0.030*"histori" + 0.027*"price"'), (23, u'0.200*"mean" + 0.094*"say" + 0.046*"die" + 0.041*"futur"'), (31, u'0.072*"sex" + 0.063*"data" + 0.038*"law" + 0.035*"big"'), (13, u'0.159*"life" + 0.104*"movi" + 0.075*"chang" + 0.040*"watch"')]


In [ ]:
###### Predict a topic
pivot = 100
print(test_buf.iloc[pivot])
print(topic_predict(test_buf.iloc[pivot], ldamodel_dump, dictionary))

In [9]:
###### Predict a topic
print("="*90)
for i in range(100,1000,100):
    pivot = i
    print("Question:\n"+test_buf.iloc[pivot]+"\n")
    print(topic_predict(test_buf.iloc[pivot], ldamodel_dump, dictionary))
    print("="*90)

Question:
Will there really be any war between India and Pakistan over the Uri attack  What will be its effects 

[u'0.127', u'"t" + 0.065', u'"want" + 0.053', u'"feel" + 0.051', u'"love" + 0.047', u'"don" + 0.041', u'"friend" + 0.028', u'"can" + 0.028', u'"like" + 0.021', u'"now" + 0.021', u'"know" + 0.021', u'"tell" + 0.020', u'"doesn" + 0.020', u'"even" + 0.016', u'"relationship" + 0.015', u'"talk" + 0.014', u'"get" + 0.014', u'"tri" + 0.013', u'"marri" + 0.012', u'"someth" + 0.012', u'"alway"']
Question:
What is the easiest way to become a billionaire($) 

[u'0.127', u'"t" + 0.065', u'"want" + 0.053', u'"feel" + 0.051', u'"love" + 0.047', u'"don" + 0.041', u'"friend" + 0.028', u'"can" + 0.028', u'"like" + 0.021', u'"now" + 0.021', u'"know" + 0.021', u'"tell" + 0.020', u'"doesn" + 0.020', u'"even" + 0.016', u'"relationship" + 0.015', u'"talk" + 0.014', u'"get" + 0.014', u'"tri" + 0.013', u'"marri" + 0.012', u'"someth" + 0.012', u'"alway"']
Question:
How can we acquire a positive mor